<a href="https://colab.research.google.com/github/alifele/Brain_Computer_Interface-Competition-/blob/master/The%20Models/Subject_based_RNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('/content/drive/My Drive/EEG_BCI/')
import datafetch
import plotters
import random

In [0]:
import random
from sklearn.model_selection import train_test_split

In [3]:
subj_1 = datafetch.fetch_subject(12)
subj_1.keys()

dict_keys(['test_data', 'train_data_class1', 'train_data_class2', 'train_data_class3', 'train_data_class4'])

In [0]:
def selector_desing(subject):
  selector = []
  for i in subject:
    for j in range(1,5):
      for k in range(18):
        selector.append((i,j,k))

  selector = np.array(selector)
  np.random.shuffle(selector)

  return selector

In [0]:
def selector_desing_test_data(subject):
  selector = []
  for i in subject:
    for j in range(1,5):
      for k in range(18,25):
        selector.append((i,j,k))

  selector = np.array(selector)
  np.random.shuffle(selector)
  return selector

In [6]:
selector = selector_desing([1])
selector.shape

(72, 3)

In [0]:
global indexes
global test_data_indexes
global selector
global selector_test_data
global under_sample_rate
under_sample_rate = 5
selector_test_data = selector_desing_test_data([1])
selector = selector_desing([1])
indexes = [i for i in range(18)]
test_data_indexes = [i for i in range(18,25)]

In [0]:
def data_loader(select):
  
  global indexes
  global under_sample_rate

  this_trail = select[2]
  index = select[2]
  
  Indexes = indexes.copy()
  Indexes.pop(index)  # so, now Indexes is the array [0,1,2,...,17] with out the this_trial index

  samples = random.sample(Indexes, 11)
  samples += [this_trail]


  data =  np.mean(subj_1['train_data_class{}'.format(select[1])][samples][::under_sample_rate,:], axis=0)
  label = select[1]
  return data, label

In [0]:
def data_loader_test_data(select):
  
  global test_data_indexes
  global under_sample_rate

  this_trail = select[2]
  index = select[2]-18
  
  Indexes = test_data_indexes.copy()

  Indexes.pop(index)  # so, now Indexes is the array [18,19,...,24] with out the this_trial index


  samples = random.sample(Indexes, 3)
  samples += [this_trail]


  data =  np.mean(subj_1['train_data_class{}'.format(select[1])][samples][::under_sample_rate,:], axis=0)
  label = select[1]
  return data, label

In [0]:
from plotters import EEG_plotter

In [0]:
#EEG_plotter(40,data)
#The meaned data

In [0]:
#EEG_plotter(40,subj_1['train_data_class{}'.format(select[1])][select[2]] )

# The Unmeaned data

In [0]:
def batch_loader(batch=[0,100]):
  # the batch=[a,b] can be 0...18000*4
  
  Label = []
  Data = []
  global selector
  

  for i in range(batch[0], batch[1]):
    data, label = data_loader(selector[int(i%len(selector))])
    Data.append(data)
    Label.append(label)

  Data = np.array(Data)
  Label = np.array(Label)

  return Data, Label

In [0]:
def batch_loader_show(label, batch=[0,100]):
  # the batch=[a,b] can be 0...18000*4
  
  Label = []
  Data = []
  global selector

  for i in range(batch[0], batch[1]):
    new_selector = selector[selector[:,1]==label[int(i%len(selector))]]
    data, label_ = data_loader(new_selector[int(i%len(new_selector))])
    Data.append(data)
    Label.append(label_)

  Data = np.array(Data)
  Label = np.array(Label)

  return Data, Label

In [0]:
#data, label = batch_loader()

In [0]:
#label, label.shape

In [0]:
def batch_loader_test_data(batch=[0,100]):
  # the batch=[a,b] can be 0...35*4
  
  Label = []
  Data = []
  global selector_test_data
  global selector
  

  for i in range(batch[0], batch[1]):
    data, label = data_loader_test_data(selector_test_data[int(i%len(selector_test_data))])
    '''
    L = len(selector[selector[:,1]==label])
    data_for_mean = []
    for j in range(6):
      data_, label_ = data_loader(selector[selector[:,1]==label][int((i+j)%L)])
      data_for_mean.append(data_)
      data = np.mean(data_for_mean, axis=0)
    '''
    Data.append(data)
    Label.append(label)

  Data = np.array(Data)
  Label = np.array(Label)

  return Data, Label

In [0]:
#data, label  = batch_loader()

In [0]:
#data.shape, label.shape

# The Model

In [0]:
from tensorflow import keras

In [0]:
model = keras.models.Sequential()

#model.add(keras.layers.MaxPool1D(2, input_shape=[None, 63]))
model.add(keras.layers.Conv1D( 3, 100, strides=50, input_shape=[None, 63], activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.GRU(30, return_sequences=True, input_shape =[None, 63]))
model.add(keras.layers.GRU(1))
model.add(keras.layers.Dense(10, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(6, activation='relu'))
model.add(keras.layers.Dense(4, activation ='softmax'))

# This got %34 accuracy

In [22]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 3)           18903     
_________________________________________________________________
dropout (Dropout)            (None, None, 3)           0         
_________________________________________________________________
gru (GRU)                    (None, None, 30)          3150      
_________________________________________________________________
gru_1 (GRU)                  (None, 1)                 99        
_________________________________________________________________
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6

In [0]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics= ['accuracy'])

In [25]:

intervals = [[3000*i,3000*(i+1)] for i in range(10)]
for epoch in range(1):
  for interval in intervals:
    data, label = batch_loader(interval)
    data_test, label_test=  batch_loader_test_data()
    model.fit(data,  label-1, epochs=20, validation_data=(data_test, label_test-1))
    print('')
    print('############################')
    print('############################')
    print('interval:{}    and    epoch:{}'.format(interval, epoch))
    print('############################')
    print('############################')
    print('')

Epoch 1/20
94/94 [==============================] - 6s 68ms/step - loss: 1.2452 - accuracy: 0.3920 - val_loss: 1.4814 - val_accuracy: 0.4200
Epoch 2/20
94/94 [==============================] - 6s 67ms/step - loss: 1.2234 - accuracy: 0.4117 - val_loss: 1.5593 - val_accuracy: 0.4500
Epoch 3/20
94/94 [==============================] - 6s 68ms/step - loss: 1.1670 - accuracy: 0.4377 - val_loss: 1.6793 - val_accuracy: 0.3900
Epoch 4/20
94/94 [==============================] - 6s 68ms/step - loss: 1.1218 - accuracy: 0.4467 - val_loss: 1.8014 - val_accuracy: 0.3900
Epoch 5/20
94/94 [==============================] - 6s 68ms/step - loss: 1.1591 - accuracy: 0.4463 - val_loss: 1.9221 - val_accuracy: 0.3000
Epoch 6/20
94/94 [==============================] - 6s 68ms/step - loss: 1.0670 - accuracy: 0.4937 - val_loss: 2.1012 - val_accuracy: 0.2400
Epoch 7/20
94/94 [==============================] - 6s 67ms/step - loss: 1.0520 - accuracy: 0.5037 - val_loss: 2.0741 - val_accuracy: 0.2300
Epoch 8/20
94

KeyboardInterrupt: ignored

In [0]:
data_show.shape

In [0]:
data_test, label_test=  batch_loader_test_data()

In [0]:
Data_test, Label_test = batch_loader()

In [0]:
model.evaluate(Data_test, Label_test-1)

#y_pred = model.predict(data_test)
#y_pred = np.argmax(y_pred, axis=1) +1

In [0]:
y_pred = model.predict(data_test)

In [0]:
np.shape(y_pred[0]), np.shape(y_pred[1])

In [0]:
plt.plot(y_pred[0][19,:,0])

In [53]:
'''
model = keras.models.Sequential()

#model.add(keras.layers.MaxPool1D(2, input_shape=[None, 63]))
#model.add(keras.layers.Conv1D( 3, 100, strides=50, input_shape=[None, 63], activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.GRU(30, return_sequences=True, input_shape =[None, 63]))
model.add(keras.layers.GRU(20))
model.add(keras.layers.Dense(10, activation = 'relu'))
model.add(keras.layers.Dense(6, activation='relu'))
model.add(keras.layers.Dense(4, activation ='softmax'))

# This got %28  accuracy

'''

"\nmodel = keras.models.Sequential()\n\n#model.add(keras.layers.MaxPool1D(2, input_shape=[None, 63]))\n#model.add(keras.layers.Conv1D( 3, 100, strides=50, input_shape=[None, 63], activation='relu'))\n#model.add(keras.layers.Dropout(0.5))\nmodel.add(keras.layers.GRU(30, return_sequences=True, input_shape =[None, 63]))\nmodel.add(keras.layers.GRU(20))\nmodel.add(keras.layers.Dense(10, activation = 'relu'))\nmodel.add(keras.layers.Dense(6, activation='relu'))\nmodel.add(keras.layers.Dense(4, activation ='softmax'))\n\n# This got %28  accuracy\n\n"

In [56]:
'''model = keras.models.Sequential()

#model.add(keras.layers.MaxPool1D(2, input_shape=[None, 63]))
#model.add(keras.layers.Conv1D( 3, 100, strides=50, input_shape=[None, 63], activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.GRU(30, return_sequences=True, input_shape =[None, 63]))
model.add(keras.layers.GRU(20))
model.add(keras.layers.Dense(10, activation = 'relu'))
model.add(keras.layers.Dense(6, activation='relu'))
model.add(keras.layers.Dense(4, activation ='softmax'))

### It Got 100% accuracy with pusing method'''

"model = keras.models.Sequential()\n\n#model.add(keras.layers.MaxPool1D(2, input_shape=[None, 63]))\n#model.add(keras.layers.Conv1D( 3, 100, strides=50, input_shape=[None, 63], activation='relu'))\n#model.add(keras.layers.Dropout(0.5))\nmodel.add(keras.layers.GRU(30, return_sequences=True, input_shape =[None, 63]))\nmodel.add(keras.layers.GRU(20))\nmodel.add(keras.layers.Dense(10, activation = 'relu'))\nmodel.add(keras.layers.Dense(6, activation='relu'))\nmodel.add(keras.layers.Dense(4, activation ='softmax'))\n\n### It Got 100% accuracy with pusing method"

In [0]:
'''## For the manual Model

over_sample = 1
intervals = [[1000*i,1000*(i+1)] for i in range(3)]
for epoch in range(1):
  for interval in intervals:

    data, label = batch_loader(interval)
    data_show, label_ = batch_loader_show(label, interval) #used as the first output
    data_test, label_test=  batch_loader_test_data()

    model.fit(data[:,::over_sample,:], (data_show, label-1), epochs=10, validation_data=(data_test[:,::over_sample,:], label_test-1))
    print('')
    print('############################')
    print('############################')
    print('interval:{}    and    epoch:{}'.format(interval, epoch))
    print('############################')
    print('############################')
    print('')'''

In [51]:
'''input_ = keras.layers.Input(shape=(None, 63), name='inputtt')
hidden1 = keras.layers.TimeDistributed(keras.layers.Dense(50, activation="relu"))(input_)
hidden2 = keras.layers.TimeDistributed(keras.layers.Dense(40, activation="relu"))(hidden1)
hidden3 = keras.layers.TimeDistributed(keras.layers.Dense(30, activation="relu"))(hidden2)
hidden3_0 = keras.layers.GRU(20)(hidden3)
hidden3_1 = keras.layers.Dense(10, activation="relu")(hidden3_0)
hidden3_2 = keras.layers.Dense(5, activation="relu")(hidden3_1)

hidden4 = keras.layers.TimeDistributed(keras.layers.Dense(40, activation="relu"))(hidden3)
hidden5 = keras.layers.TimeDistributed(keras.layers.Dense(63, activation="relu"))(hidden4)
output = keras.layers.Dense(1, name="time_series_output")(hidden5)
aux_output = keras.layers.Dense(4, activation='softmax', name="label_output")(hidden3_2)
model = keras.Model(inputs=[input_], outputs=[output,
aux_output])



keras.utils.plot_model(model)
from IPython.display import Image
Image('model.png')


model.compile(optimizer='adam', 
              loss = ['MSE', 'sparse_categorical_crossentropy'],
              metrics = ['accuracy'])

model.summary()'''

'input_ = keras.layers.Input(shape=(None, 63), name=\'inputtt\')\nhidden1 = keras.layers.TimeDistributed(keras.layers.Dense(50, activation="relu"))(input_)\nhidden2 = keras.layers.TimeDistributed(keras.layers.Dense(40, activation="relu"))(hidden1)\nhidden3 = keras.layers.TimeDistributed(keras.layers.Dense(30, activation="relu"))(hidden2)\nhidden3_0 = keras.layers.GRU(20)(hidden3)\nhidden3_1 = keras.layers.Dense(10, activation="relu")(hidden3_0)\nhidden3_2 = keras.layers.Dense(5, activation="relu")(hidden3_1)\n\nhidden4 = keras.layers.TimeDistributed(keras.layers.Dense(40, activation="relu"))(hidden3)\nhidden5 = keras.layers.TimeDistributed(keras.layers.Dense(63, activation="relu"))(hidden4)\noutput = keras.layers.Dense(1, name="time_series_output")(hidden5)\naux_output = keras.layers.Dense(4, activation=\'softmax\', name="label_output")(hidden3_2)\nmodel = keras.Model(inputs=[input_], outputs=[output,\naux_output])\n\n\n\nkeras.utils.plot_model(model)\nfrom IPython.display import Image

ERROR! Session/line number was not unique in database. History logging moved to new session 63


In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
mat = confusion_matrix(label_test, y_pred)

In [0]:
plt.imshow(mat)
mat


In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(label_test, y_pred))